In [ ]:
from datetime import datetime
import pandas as pd
from datetime import timezone

In [ ]:
start_date = datetime.fromisoformat("2018-01-01 00:00").astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
end_date = datetime.fromisoformat("2018-12-31 23:59").astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()

In [ ]:
apple_news = pd.read_pickle("pkls/apple_news_data.pkl", compression='infer')
amazon_news = pd.read_pickle("pkls/amazon_news_data.pkl", compression='infer')
apple_chart = pd.read_pickle("pkls/apple_stock_chart.pkl", compression='infer')
amazon_chart = pd.read_pickle("pkls/apple_stock_chart.pkl", compression='infer')

In [ ]:
apple_news

In [ ]:
amazon_news

In [ ]:
apple_chart["articles"] = pd.NaT

In [ ]:
amazon_chart

In [90]:
for index, row in apple_chart.iterrows():
    time = datetime.fromtimestamp(row['timestamp']).date()
    articles=[]
    for news_index,news_row in apple_news.iterrows():
        if datetime.fromtimestamp(news_row['published_time']).date() > time:
            break
        if datetime.fromtimestamp(news_row['published_time']).date() == time:
            articles.append(news_row['article_content'])
    apple_chart.at[index, 'articles'] = articles

In [93]:
apple_chart.reset_index(inplace=True)
apple_chart

,index,timestamp,trends,articles
0,870,1.514876e+09,1,[what happened shares of many optical networki...
1,871,1.514963e+09,0,"[allergan plans to cut more than 1,000 jobs am..."
2,872,1.515049e+09,1,[longtime music executive jimmy iovine plans t...
3,873,1.515136e+09,1,"[by sara sjolin, marketwatch\nwages are key pa..."
4,874,1.515395e+09,0,[2 big shareholders want apple to investigate ...
...,...,...,...,...
246,1116,1.545635e+09,0,[stock market power rankings: youtube catches ...
247,1117,1.545808e+09,1,"[wednesday’s wild day: 504 stocks rose, and on..."
248,1118,1.545894e+09,1,[print\nstock markets staged a miraculous come...
249,1119,1.545980e+09,0,[get the daily marketslant newsletter from pan...


In [98]:
len(apple_chart['articles'][30])

182